In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# General imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import seaborn as sns
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import confusion_matrix
import cv2
import PIL

In [7]:

def visualize_training_results(history):
    '''
    From https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
    
    Input: keras history object (output from trained model)
    '''
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    fig.suptitle('Model Results')

    # summarize history for accuracy
    ax1.plot(history.history['acc'])
    ax1.plot(history.history['val_acc'])
    ax1.set_ylabel('Accuracy')
    ax1.legend(['train', 'test'], loc='upper left')
    # summarize history for loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_ylabel('Loss')
    ax2.legend(['train', 'test'], loc='upper left')
    
    plt.xlabel('Epoch')
    plt.show()

In [17]:
# Set up ImageDataGenerator
train_imagegen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   zoom_range=([0.6,1]),
                                   rotation_range=10,                           
                                   brightness_range=([0.6, 1.5]),
                                   horizontal_flip=True,
                                   validation_split=0.06) # this will set aside a part of training set for validation data
test_imagegen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   zoom_range=([0.6,1]),
                                   rotation_range=10,
                                   brightness_range=([0.6,1.5]),
                                   horizontal_flip=True)
# Bring the data in
train_generator = train_imagegen.flow_from_directory(
                                    '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training',
                                    target_size=(200,200),
                                    batch_size=20,
                                    seed=42,
                                    class_mode='binary',
                                    subset='training')

test_generator = test_imagegen.flow_from_directory(
                                    '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Testing',
                                    target_size=(200,200),
                                    batch_size=20,
                                    seed=42,
                                    class_mode='binary')

val_generator = train_imagegen.flow_from_directory(
                                    '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training',
                                    target_size=(200,200),
                                    batch_size=20,
                                    seed=42,
                                    class_mode='binary',
                                    subset='validation')

In [18]:
# Visualize (code from https://github.com/austint1121/OES-PneumoniaClassification/blob/main/Final_Notebook.ipynb)
train_batch = train_generator.next()
fig, axes = plt.subplots(2, 5, figsize=(16, 8))
    
for i in range(10):
    # Load image into numpy array and re-scale
    img = np.array(train_batch[0][i] * 255, dtype='uint8')
    ax = axes[i // 5, i % 5]
    ax.imshow(img)
fig.suptitle('Training Images')
plt.tight_layout()
plt.show()

## **Taking a Look at a few different individual images**

In [13]:
tumor1 = PIL.Image.open('../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training/AllTumorsTrain/gg (108).jpg')
tumor1

In [ ]:
# Figuring out the number of color channels
tumor1.mode

## **Taking a look at the class imbalance**

In [15]:
train_generator.class_indices

In [16]:
train_generator.classes

In [17]:
tumors = pd.DataFrame(train_generator.classes)
values = tumors.value_counts()
values

In [18]:
tumors.rename(columns={0:'Tumor/No Tumor'}, inplace=True)

In [19]:
tumors[tumors['Tumor/No Tumor'] == 0]

In [20]:
no_tumor = len(tumors[tumors['Tumor/No Tumor'] == 1])
tumor = len(tumors[tumors['Tumor/No Tumor'] == 0])

In [21]:
#plt.figure(figsize=(10,6))
#sns.set(font_scale=1.4)
#sns.barplot(tumors.index, tumors.values)
#plt.ylabel("Number of Images")
#plt.title('Distribution of Brain MRIs with and without Tumor');

fig, ax = plt.subplots(figsize=(10,8))
ax.bar(x=['No Tumor', 'Tumor'],height = [no_tumor, tumor])
ax.set(xlabel='', ylabel='Number of Images', title='Distribution of Brain MRIs with and without Tumor');

## **Baseline CNN Model**

In [19]:
# Building the first baseline model; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
baseline = keras.Sequential()
baseline.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
baseline.add(layers.MaxPooling2D(2,2))
baseline.add(layers.Conv2D(64, (3,3), activation='relu'))
baseline.add(layers.MaxPooling2D(2,2))

baseline.add(layers.Flatten())
baseline.add(layers.Dense(128, activation='relu'))
baseline.add(layers.Dense(1, activation='sigmoid'))

baseline.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

In [20]:
baseline_results = baseline.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)

In [21]:
visualize_training_results(baseline_results)

### **Analysis of Model**

Looking at the above graphs, it is obvious that this first baseline cnn model is overfitting; accuracy for training data ends up at around 94%, whereas testing data ends up at around 65%. Additionally, the loss for testing data is fairly high; for training the loss ends up at 15%, and for testing it ends up at 69%. In the next model iteration, I will add another dense layer, which will hopefully help the model pick up on more patterns, and some dropout layers for a form of regularization.

## **Adding another Dense layer and Dropout layers**

In [22]:
# Adding another dense layer and a couple of dropout layers; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
np.random.seed(42)
layers_drop = keras.Sequential()
layers_drop.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
layers_drop.add(layers.MaxPooling2D(2,2))
layers_drop.add(layers.Conv2D(64, (3,3), activation='relu'))
layers_drop.add(layers.MaxPooling2D(2,2))

layers_drop.add(layers.Flatten())
layers_drop.add(layers.Dense(128, activation='relu'))
layers_drop.add(layers.Dropout(0.3))
layers_drop.add(layers.Dense(64, activation='relu'))
layers_drop.add(layers.Dropout(0.3))
layers_drop.add(layers.Dense(1, activation='sigmoid'))

layers_drop.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

In [23]:
# Fitting the model
layers_drop_results = layers_drop.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)

In [24]:
visualize_training_results(layers_drop_results)

### **Analysis of Model**

In this iteration, training accuracy ends up at 93%, and testing ends up at 73%, so the model is overfitting, but less so than the baseline model. As for loss, training loss is 14% and testing loss is 65%, which is not drastically different from the last model. Adding another layer and dropout layers did not seem to help very much, but in the next model iteration I am going to account for the class imabalance, and the added layer and dropout layers might perform better in this iteration.

## **Accounting for class imbalance**

In [25]:
# Accounting for class imbalance; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/

class_ld = keras.Sequential()
class_ld.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_ld.add(layers.MaxPooling2D(2,2))
class_ld.add(layers.Conv2D(64, (3,3), activation='relu'))
class_ld.add(layers.MaxPooling2D(2,2))

class_ld.add(layers.Flatten())
class_ld.add(layers.Dense(128, activation='relu'))
class_ld.add(layers.Dropout(0.3))
class_ld.add(layers.Dense(64, activation='relu'))
class_ld.add(layers.Dropout(0.3))
class_ld.add(layers.Dense(1, activation='sigmoid'))

class_ld.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights = {0: 1, # TUMOR
          1:6.255} # NO TUMOR 
# there are 6.255 times as many images of MRIs with tumors than without

In [26]:
# Fitting the model
class_ld_results = class_ld.fit_generator(train_generator,
                                          class_weight=weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)

In [27]:
visualize_training_results(class_ld_results)

### **Analysis of Model**

In this model iteration, training accuracy was about 89% and testing accuracy is about 57%, so the model is still overfitting compared to the last model. Loss for training is at around 51% and testing loss is around 99%. In terms of acuracy and loss, the model is doing worse than the previous model. However, recall has increased significantly, so it seems that adding weights is beneficial to the model, even though it requires further tuning.

## **Adding another Convolution layer**

In [28]:
# Another Convolution layer; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
np.random.seed(42)
class_con = keras.Sequential()
class_con.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_con.add(layers.MaxPooling2D(2,2))
class_con.add(layers.Conv2D(64, (3,3), activation='relu'))
class_con.add(layers.MaxPooling2D(2,2))
class_con.add(layers.Conv2D(128, (3,3), activation='relu'))
class_con.add(layers.MaxPooling2D(2,2))

class_con.add(layers.Flatten())
class_con.add(layers.Dense(128, activation='relu'))
class_con.add(layers.Dropout(0.3))
class_con.add(layers.Dense(64, activation='relu'))
class_con.add(layers.Dropout(0.3))
class_con.add(layers.Dense(1, activation='sigmoid'))

class_con.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights = {0: 1, 
          1:6.255}

In [29]:
class_con_results = class_con.fit_generator(train_generator,
                                          class_weight=weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)

In [31]:
visualize_training_results(class_con_results)

### **Analysis of Model**

Training accuracy is at 89% while testing accuracy is at 50%, so model is still overfitting, slightly worse than the previous model. Training loss is 50%, and testing loss is 93%; training loss is 50% and testing loss is 93%, which is similar to the loss values of the previous model. In the next model iteration, I will see if changing the dimensions of the pooling layer will improve the model.


## **Adjusting the Pooling Strategy**

In [32]:
# Another Convolution layer; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
np.random.seed(42)
class_pool = keras.Sequential()
class_pool.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_pool.add(layers.MaxPooling2D(2,2))
class_pool.add(layers.Conv2D(64, (3,3), activation='relu'))
class_pool.add(layers.MaxPooling2D(3,3))
class_pool.add(layers.Conv2D(128, (3,3), activation='relu'))
class_pool.add(layers.MaxPooling2D(5,5))

class_pool.add(layers.Flatten())
class_pool.add(layers.Dense(128, activation='relu'))
class_pool.add(layers.Dropout(0.3))
class_pool.add(layers.Dense(64, activation='relu'))
class_pool.add(layers.Dropout(0.3))
class_pool.add(layers.Dense(1, activation='sigmoid'))

class_pool.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights = {0: 1, # TUMOR
          1:6.255} # NO TUMOR
# there are 6.255 times as many images of MRIs with tumors than without

In [33]:
class_pool_results = class_pool.fit_generator(train_generator,
                                          class_weight=weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)

In [34]:
visualize_training_results(class_pool_results)

### **Analysis of Model**

Training accuracy is at about 89%, and testing accuracy is at about 48%, so model is still overfitting quite a bit. Training loss is 48%, and testing loss is 101%; testing loss has increased quite a lot since the last model. Maybe increasing the pooling matrix is not beneficial to the model, but maybe padding will help. In this next model, I will introduce some padding to reduce image loss, to see if this improves model. I will also increase the number of epochs.

## **Model with Padding, More Epochs**

In [35]:
# Another Convolution layer; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
np.random.seed(42)
class_pad = keras.Sequential()
class_pad.add(layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(200,200,3)))
class_pad.add(layers.MaxPooling2D(2,2))
class_pad.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
class_pad.add(layers.MaxPooling2D(3,3))
class_pad.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
class_pad.add(layers.MaxPooling2D(5,5))

class_pad.add(layers.Flatten())
class_pad.add(layers.Dense(128, activation='relu'))
class_pad.add(layers.Dropout(0.3))
class_pad.add(layers.Dense(64, activation='relu'))
class_pad.add(layers.Dropout(0.3))
class_pad.add(layers.Dense(1, activation='sigmoid'))

class_pad.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights = {0: 1, # TUMOR
          1:6.255} # NO TUMOR
# there are 6.255 times as many images of MRIs with tumors than without

In [36]:
early_stop = [EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
            ModelCheckpoint(filepath='best_model.h5', monitor='val_loss',
                           save_best_only=True)]

In [39]:
class_pad_results = class_pad.fit_generator(train_generator,
                                          class_weight=weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=50,
                                         validation_data=test_generator)

In [40]:
visualize_training_results(class_pad_results)

### **Analysis of Model**

Training accuracy is 96%, testing accuracy is 68%, so the model is overfitting a bit less than in the previous model. Training loss is 29% and testing loss is 78%; testing loss increased significantly from the last model. It looks like this strategy of padding is not very good for this data.

## **Model without Padding, Early Stopping, and more Epochs**

In [41]:
# Another Convolution layer; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
np.random.seed(42)
class_ee = keras.Sequential()
class_ee.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_ee.add(layers.MaxPooling2D(2,2))
class_ee.add(layers.Conv2D(64, (3,3), activation='relu'))
class_ee.add(layers.MaxPooling2D(3,3))
class_ee.add(layers.Conv2D(128, (3,3), activation='relu'))
class_ee.add(layers.MaxPooling2D(5,5))

class_ee.add(layers.Flatten())
class_ee.add(layers.Dense(128, activation='relu'))
class_ee.add(layers.Dropout(0.3))
class_ee.add(layers.Dense(64, activation='relu'))
class_ee.add(layers.Dropout(0.3))
class_ee.add(layers.Dense(1, activation='sigmoid'))

class_ee.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights = {0: 1, # TUMOR
          1:6.255} # NO TUMOR
# there are 6.255 times as many images of MRIs with tumors than without

In [42]:
early_stop2 = [EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
            ModelCheckpoint(filepath='best_model.h5', monitor='val_loss',
                           save_best_only=True)]

In [43]:
class_ee_results = class_ee.fit_generator(train_generator,
                                          class_weight=weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=50,
                                        callbacks=early_stop2,
                                         validation_data=test_generator)

In [44]:
visualize_training_results(class_ee_results)